In [340]:
import math
import numpy as np

In [341]:
R = 100 #Ohm
L = 10*1e-9 #H
f = 1e9 #GHz
W = 2*math.pi*f
ZL = R + 2*math.pi*f*L*1J
print("ZL = {}".format(ZL))
Zo = 50
#Asumiendo vacío
lambda_z = 3*1e8/f
print(lambda_z)

ZL = (100+62.83185307179586j)
0.3


## Diseño Red L

Dado que R > Zo, se aplican las siguientes fórmulas para buscar las susceptancias y reactancias:
$B = \frac{X_L \pm \sqrt{\frac{R_L}{Z_o}}\cdot \sqrt{R_L^2 + X_L^2Z_oR_L}}{R_L^2 + X_L^2}$
$X = \frac{1}{B} + \frac{X_LZ_o}{R_L} + \frac{Z_o}{BR_L}$

In [342]:
B_1 = lambda RL, XL: [(XL+(np.sqrt(RL/Zo)*np.sqrt(RL**2 + XL**2 - (Zo*RL))))/(RL**2 + XL**2), (XL-(np.sqrt(RL/Zo)*np.sqrt(RL**2 + XL**2 - (Zo*RL))))/(RL**2 + XL**2)]
X_1 = lambda RL, XL, B: [(1/B[0]) + ((XL*Zo)/RL) - (Zo/(RL*B[0])), (1/B[1]) + ((XL*Zo)/RL) - (Zo/(RL*B[1]))]
B = B_1(ZL.real, ZL.imag)
X = X_1(ZL.real, ZL.imag, B_1(ZL.real, ZL.imag))
print("solución 1: B = {}, X = {}".format(B[0], X[0]))
print("solución 2: B = {}, X = {}".format(B[1], X[1]))

solución 1: B = 0.01409584404804315, X = 66.887374595045
solución 2: B = -0.005086299180675378, X = -66.88737459504497


In [343]:
rta_redesL = []
for B_ in B:
    if (B_ > 0):
        rta_redesL.append( [B_, "Capacitor Paralelo", B_/W] )
    else:
        rta_redesL.append( [B_,  "Inductor Paralelo", -1/(B_*W)] )
for X_ in X:
    if (X_ > 0):
        rta_redesL.append( [X_, "Inductor Serie", X_/W] )
    else:
        rta_redesL.append( [X_,  "Capacitor Serie", -1/(X_*W)] )
Cp, Cs, Lp, Ls = 0, 0, 0, 0
print("Solución 1:")
if(rta_redesL[0][1] == "Capacitor Paralelo"):
    print(rta_redesL[0][1] + " Cp = {}pF".format(round(rta_redesL[0][2]*1e12, 3)))
    Cp = rta_redesL[0][2]
else:
    print(rta_redesL[0][1] + " Lp = {}nH".format(round(rta_redesL[0][2]*1e9, 3)))
    Lp = rta_redesL[0][2]
if(rta_redesL[2][1] == "Capacitor Serie"):
    print(rta_redesL[2][1] + " Cs = {}pF".format(round(rta_redesL[2][2]*1e12, 3)))
    Cs = rta_redesL[2][2]
else:
    print(rta_redesL[2][1] + " Ls = {}nH".format(round(rta_redesL[2][2]*1e9, 3)))
    Ls = rta_redesL[2][2]
print("Solución 2:")
if(rta_redesL[1][1] == "Capacitor Paralelo"):
    print(rta_redesL[1][1] + " Cp = {}pF".format(round(rta_redesL[1][2]*1e12, 3)))
    Cp = rta_redesL[1][2]
else:
    print(rta_redesL[1][1] + " Lp = {}nH".format(round(rta_redesL[1][2]*1e9, 3)))
    Lp = rta_redesL[1][2]
if(rta_redesL[3][1] == "Capacitor Serie"):
    print(rta_redesL[3][1] + " Cs = {}pF".format(round(rta_redesL[3][2]*1e12, 3)))
    Cs = rta_redesL[3][2]
else:
    print(rta_redesL[3][1] + " Ls = {}nH".format(round(rta_redesL[3][2]*1e9, 3)))
    Ls = rta_redesL[3][2]

Solución 1:
Capacitor Paralelo Cp = 2.243pF
Inductor Serie Ls = 10.645nH
Solución 2:
Inductor Paralelo Lp = 31.291nH
Capacitor Serie Cs = 2.379pF


Para el cálculo de la ROE se tiene lo siguiente:
\begin{align*}
&\text{Opción 1:}&
&\text{Opción 2:}\\
&Z_{in} = 10.65\text{nH}\cdot \omega j + \left ( Z_L\;//\;\frac{-j}{2.24\text{pF}\cdot\omega} \right )&
&Z_{in} = \frac{-j}{2.4\text{pF}\cdot\omega} + \left ( Z_L\;//\;31,3\text{nH}\cdot \omega j  \right )\\
\end{align*}

In [344]:
RedL_Zin_1 = lambda f: Ls*2*math.pi*f*1J + ((1/ZL) + (-1J/(Cp*2*math.pi*f))**-1)**-1
RedLZin_2 = lambda f: (-1J/(Cs*2*math.pi*f)) + ((1/ZL) + (Lp*2*math.pi*f*1J)**-1)**-1
#Chequeo que de ~50 Ohms a 1GHz
print(RedL_Zin_1(1e9), RedLZin_2(1e9))

(49.99999999999999+0j) (49.99999999999999+2.842170943040401e-14j)


## Diseño Stub Simple

Para el cálculo del Stub se aplican las siguiente fórmulas:
\begin{equation*}
    t = \left\{\begin{matrix}
\left ( X_L\pm \sqrt{\frac{R_L[(Z_o - R_L)^2+X_L^2]}{Z_o}} \right )\frac{1}{R_L-Z_o}\;\;\;\; R_L\neq Z_o\\
-\frac{X_L}{2Z_o}\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\; R_L= Z_o
\end{matrix}\right.
\end{equation*}
La distancia entre la carga y el stub se calcula como:
\begin{equation*}
d = \left\{\begin{matrix}
\frac{\lambda}{2\pi}\cdot arctg(t)\;\;\;\;\;\;\;\;\; t \geq 0\\\\
\frac{\lambda}{2\pi}[\pi + arctg(t)]\;\;\; t < 0
\end{matrix}\right.
\end{equation*}
Luego las longitudes del stub se calcularan de forma gráfica con el programa $\textit{Smith V4}$.

In [345]:
t_stub_par = lambda RL, XL: [(XL+np.sqrt(RL*((Zo-RL)**2 + XL**2)/Zo))/(RL-Zo) , (XL-np.sqrt(RL*((Zo-RL)**2 + XL**2)/Zo))/(RL-Zo) ]
d_stub_par = lambda t: np.arctan(t)/(2*math.pi) if (t >= 0) else (1/(2*math.pi))*(math.pi+np.arctan(t))
t1 = t_stub_par(ZL.real, ZL.imag)
print("Valores de t = {}".format([t1[0],t1[1]]))
print("Distancias (en función de lambda):")
print("d1 = {} λ, d2 = {} λ".format(round(d_stub_par(t1[0]),3), round(d_stub_par(t1[1]),3)))

Valores de t = [3.5278203231628513, -1.0145462002910168]
Distancias (en función de lambda):
d1 = 0.206 λ, d2 = 0.374 λ


La longitud de las lineas puede calcularse de la siguiente forma:
\begin{align*}
\end{align*}